In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

## Goal
Build a simulation of robot moving on 2D grid. Apply discrete Bayesian filter to filter it's position and draw fancy plots.

In [2]:
from ipywidgets import interact, Button
import ipywidgets as widgets
from IPython.display import clear_output

In [3]:
plot_out = widgets.Output()
log_out = widgets.Output()

In [4]:
class Robot:
    def __init__(self, x=0, y=0, xmax=10, ymax=10, error_rate=0.1):
        self.x = x
        self.y = y
        self.xmax = xmax
        self.ymax = ymax
        self.error_rate = error_rate
    
    def move(self, dx=0, dy=0):
        self.x += dx
        self.y += dy
        self.x, self.y = self._constraint(self.x, self.y)
    
    def _constraint(self, x=None, y=None):
        if x < 0:
            x = self.xmax + x
        elif x >= self.xmax:
            x = 0
        if y < 0:
            y = self.ymax + y
        elif y >= self.ymax:
            y = 0
        return x, y
    
    def measure(self):
        """ Measurement of robot position with error """
        x, y = self.x, self.y
        if np.random.rand() < self.error_rate:
            # Error occurred
            if np.random.rand() < .5:
                # Error in x
                if np.random.rand() < .5:
                    x += 1
                else:
                    x -= 1
            else:
                # Error in y
                if np.random.rand() < .5:
                    y += 1
                else:
                    y -= 1
        return self._constraint(x, y)
                
                    
    
    @property
    def place(self):
        return self.x, self.y

In [137]:
np.linspace(0, 9, 91)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4,
       6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
       7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. ])

In [148]:
w, h = 10, 10
x = np.linspace(0, w - 1, num=91)
y = np.linspace(0, h - 1, num=91)
xx, yy = np.meshgrid(x, y)
xx2, yy2 = np.meshgrid(np.arange(w), np.arange(h))
rob = Robot(x=5, y=5, xmax=w, ymax=h)
filt = Filter()
filt.belief[rob.x, rob.y] = 1
filt.belief /= filt.belief.sum()

def show_pos():
    with log_out:
        print(rob.place)

def show_plot(prediction):
    x_true, y_true = rob.place
    x_measured, y_measured = rob.measure()
    
    zz = (yy==y_true) & (xx == x_true)
    zz2 = (yy==y_measured) & (xx==x_measured)
    with plot_out:
        clear_output()
        fig, ax = plt.subplots()
        ax.contour(xx, yy, zz, levels=1, cmap=plt.cm.Blues_r, alpha=.75)
        ax.contour(xx, yy, zz2, levels=1, cmap=plt.cm.Reds_r, alpha=.75)
        ax.contourf(xx2, yy2, prediction.T, alpha=.75)
        ax.legend([Line2D([0], [0], color="blue", lw=3),
                   Line2D([0], [0], color="red", lw=3)], ['True position', "Measured position"])
        plt.show()
        filt.update(x_measured, y_measured)

def left_clicked(_):
    rob.move(dx=-1)
    show_pos()
    show_plot(filt.predict(dx=1)) 

def right_clicked(_):
    rob.move(dx=1)
    show_pos()
    show_plot(filt.predict(dx=-1)) 
    

def down_clicked(_):
    rob.move(dy=-1)
    show_pos()
    show_plot(filt.predict(dy=1)) 

def up_clicked(_):
    rob.move(dy=1)
    show_pos()
    show_plot(filt.predict(dy=-1)) 
    

In [152]:
lb = widgets.Button(description="Left")
lb.on_click(left_clicked)

rb = widgets.Button(description="Right")
rb.on_click(right_clicked)

ub = widgets.Button(description="Up")
ub.on_click(up_clicked)

db = widgets.Button(description="Down")
db.on_click(down_clicked)
out = widgets.Output()

first_row = widgets.HBox([lb, rb])
second_row = widgets.HBox([ub, db])
with log_out:
    clear_output()
widgets.VBox([first_row, second_row, plot_out, log_out])

In [119]:
from scipy.signal import convolve2d

In [151]:
class Filter:
    def __init__(self):
        self.belief = np.ones((h, w)) / (h * w)
        self.gauss_kernel = np.array([[0, 0.1, 0],
                                      [.1, .6, .1],
                                      [0, .1, 0]])
    
    def predict(self, dx=0, dy=0, kernel=None):
        rolled = np.roll(np.roll(self.belief, dx, axis=1), -dy, axis=0)
        return convolve2d(rolled, self.gauss_kernel, mode='same', boundary='wrap')
    
    def likelihood(self, x, y, scale=3):
        L = self.belief[:, :]
        L[x, y] += .6
        L[x-1:x+1, y-1:y+1] += 0.1
        return L / L.sum()
    
    def update(self, x, y):
        P = self.belief * self.likelihood(x, y)
        self.belief = P / P.sum()
    
    def most_likely_position(self):
        return np.argmax(self.belief, axis=0).max(), np.argmax(self.belief, axis=1).max()

In [94]:
f = Filter()
rob = Robot(5, 5)
priors = []
for i in range(23):
    rob.move(1, 0)
    predictor = f.predict(1,0, kernel=np.array([[.1, .8, .1]]))
    priors.append(predictor)
    measure = rob.measure()
    print(f"Rob place = {rob.place} | measured = {measure}")
    f.update(*measure)


Rob place = (6, 5) | measured = (6, 5)
Rob place = (7, 5) | measured = (7, 5)
Rob place = (8, 5) | measured = (8, 5)
Rob place = (9, 5) | measured = (9, 5)
Rob place = (0, 5) | measured = (0, 5)
Rob place = (1, 5) | measured = (1, 5)
Rob place = (2, 5) | measured = (2, 5)
Rob place = (3, 5) | measured = (3, 5)
Rob place = (4, 5) | measured = (4, 5)
Rob place = (5, 5) | measured = (5, 5)
Rob place = (6, 5) | measured = (6, 6)
Rob place = (7, 5) | measured = (7, 5)
Rob place = (8, 5) | measured = (8, 5)
Rob place = (9, 5) | measured = (9, 5)
Rob place = (0, 5) | measured = (0, 5)
Rob place = (1, 5) | measured = (2, 5)
Rob place = (2, 5) | measured = (2, 5)
Rob place = (3, 5) | measured = (3, 5)
Rob place = (4, 5) | measured = (4, 5)
Rob place = (5, 5) | measured = (5, 5)
Rob place = (6, 5) | measured = (6, 5)
Rob place = (7, 5) | measured = (7, 4)
Rob place = (8, 5) | measured = (8, 5)


In [95]:
from ipywidgets import IntSlider

In [96]:
def simulate(time_step):
    plt.contourf(xx2, yy2, priors[time_step])
    plt.show()
#     _, ax = plt.subplots(ncols=3, figsize=(16, 10))
#     ax[0].contourf(xx2, yy2, prior);
#     ax[1].contourf(xx2, yy2, predictor)
#     ax[2].contourf(xx2, yy2, f.likelihood(5, 5))

In [97]:
interact(simulate, time_step=IntSlider(value=0, max=len(priors)-1));

interactive(children=(IntSlider(value=0, description='time_step', max=22), Output()), _dom_classes=('widget-in…

<function __main__.simulate(time_step)>